<picture>
  <source media="(prefers-color-scheme: dark)" srcset="https://vespa.ai/assets/vespa-ai-logo-heather.svg">
  <source media="(prefers-color-scheme: light)" srcset="https://vespa.ai/assets/vespa-ai-logo-rock.svg">
  <img alt="#Vespa" width="200" src="https://vespa.ai/assets/vespa-ai-logo-rock.svg" style="margin-bottom: 25px;">
</picture>

# Feeding to Vespa Cloud

Our previous notebook illustrated one way of benchmarking feed performance to a local Vespa instance running in Docker.
In this notebook, we will llok at the same methods, but how feeding to [Vespa Cloud](https://cloud.vespa.ai) affects performance of the different methods.

The key difference between feeding to a local Vespa instance and a Vespa Cloud instance is the network latency.
Additionally, we will introduce embedding in Vespa at feed time, which is a realistic scenario for many use-cases.

We will look at these 4 different methods:

1. Using `VespaSync`.
2. Using `VespaAsync`.
3. Using `feed_iterable()`
4. Using [Vespa CLI](https://docs.vespa.ai/en/vespa-cli)


<div class="alert alert-info">
    Refer to <a href="https://pyvespa.readthedocs.io/en/latest/troubleshooting.html">troubleshooting</a>
    for any problem when running this guide.
</div>


Install [Vespa CLI](https://docs.vespa.ai/en/vespa-cli.html).
The `vespacli` python package is just a thin wrapper, allowing for installation through pypi.

> Do NOT install if you already have the Vespa CLI installed.


In [5]:
# !pip3 install vespacli

[Install pyvespa](https://pyvespa.readthedocs.io/), other dependencies, and start the Docker Daemon.


In [6]:
# !pip3 install pyvespa datasets plotly>=5.20

## Create an application package

The [application package](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.ApplicationPackage)
has all the Vespa configuration files.

For this demo, we will use a simple application package


In [25]:
from vespa.package import (
    ApplicationPackage,
    Field,
    Schema,
    Document,
    FieldSet,
    HNSW,
)

# Define the application name (can NOT contain `_` or `-`)

application = "feedperformancecloud"


package = ApplicationPackage(
    name=application,
    schema=[
        Schema(
            name="doc",
            document=Document(
                fields=[
                    Field(name="id", type="string", indexing=["summary"]),
                    Field(name="text", type="string", indexing=["index", "summary"]),
                    Field(
                        name="embedding",
                        type="tensor<float>(x[1024])",
                        # Note that we are NOT embedding with a vespa model here, but that is also possible.
                        indexing=["summary", "attribute", "index"],
                        ann=HNSW(distance_metric="angular"),
                    ),
                ]
            ),
            fieldsets=[FieldSet(name="default", fields=["text"])],
        )
    ],
)

Note that the `ApplicationPackage` name cannot have `-` or `_`.


## Deploy the Vespa application

Deploy `package` on the local machine using Docker,
without leaving the notebook, by creating an instance of
[VespaDocker](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.deployment.VespaDocker). `VespaDocker` connects
to the local Docker daemon socket and starts the [Vespa docker image](https://hub.docker.com/r/vespaengine/vespa/).

If this step fails, please check
that the Docker daemon is running, and that the Docker daemon socket can be used by clients (Configurable under advanced settings in Docker Desktop).


In [ ]:
# import os
# import requests

# res = requests.get(
#     url="https://api.github.com/repos/vespa-engine/vespa/releases/latest"
# ).json()
# os.environ["VERSION"] = res["tag_name"].replace("v", "")
# !curl -fsSL https://github.com/vespa-engine/vespa/releases/download/v${VERSION}/vespa-cli_${VERSION}_linux_amd64.tar.gz | tar -zxf -
# !ln -sf /content/vespa-cli_${VERSION}_linux_amd64/bin/vespa /bin/vespa

## Configure data-plane security

Create Vespa Cloud data-plane mTLS cert/key-pair. This mutual certificate pair is used to talk to your Vespa cloud endpoints.
See [Vespa Cloud Security Guide](https://cloud.vespa.ai/en/security/guide).

We save the paths to the credentials, for later dataplane access without using pyvespa APIs - see example at the end of this notebook.


In [4]:
import os

os.environ["TENANT_NAME"] = "vespa-team"  # Replace with your tenant name
vespa_cli_command = (
    f'vespa config set application {os.environ["TENANT_NAME"]}.{application}'
)

!vespa config set target cloud
!{vespa_cli_command}
!vespa auth cert -N

Success: Certificate written to '/Users/thomas/.vespa/vespa-team.feedperformancecloud.default/data-plane-public-cert.pem'
Success: Private key written to '/Users/thomas/.vespa/vespa-team.feedperformancecloud.default/data-plane-private-key.pem'


In [5]:
from os.path import exists
from pathlib import Path

cert_path = (
    Path.home()
    / ".vespa"
    / f"{os.environ['TENANT_NAME']}.{application}.default/data-plane-public-cert.pem"
)
key_path = (
    Path.home()
    / ".vespa"
    / f"{os.environ['TENANT_NAME']}.{application}.default/data-plane-private-key.pem"
)

if not exists(cert_path) or not exists(key_path):
    print(
        "ERROR: set the correct paths to security credentials. Correct paths above and rerun until you do not see this error"
    )

Note that the subsequent deploy-call below will add `data-plane-public-cert.pem` to the application before deploying it to Vespa Cloud, so that
you have access to both the private key and the public certificate, while Vespa Cloud only knows the public certificate.

## Configure control-plane security

Authenticate to generate a tenant level control-plane API key for deploying the applications to Vespa Cloud, and save the path to it.

<div style="background-color: #ffcc00; padding: 10px; border: 1px solid #ff9900; font-weight: bold;">
    <strong>Warning:</strong>The generated tenant api key must be added in the Vespa Console before attempting to deploy the application.
</div>

The following step will print the following message:

```
To use this key in Vespa Cloud click 'Add custom key' at
https://console.vespa-cloud.com/tenant/TENANT_NAME/account/keys
and paste the entire public key including the BEGIN and END lines.
```


In [ ]:
!vespa auth api-key

In [7]:
from pathlib import Path

api_key_path = Path.home() / ".vespa" / f"{os.environ['TENANT_NAME']}.api-key.pem"

Follow the instrauctions from the output above and add the control-plane key in the console at `https://console.vespa-cloud.com/tenant/TENANT_NAME/account/keys`
(replace TENANT_NAME with your tenant name).


In [26]:
from vespa.deployment import VespaCloud
import os


def read_secret():
    """Read the API key from the environment variable. This is
    only used for CI/CD purposes."""
    t = os.getenv("VESPA_TEAM_API_KEY")
    if t:
        return t.replace(r"\n", "\n")
    else:
        return t


vespa_cloud = VespaCloud(
    tenant=os.environ["TENANT_NAME"],
    application=application,
    key_content=read_secret() if read_secret() else None,
    key_location=api_key_path,
    application_package=package,
)

`app` now holds a reference to a [VespaCloud](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.deployment.VespaCloud) instance.


In [27]:
app = vespa_cloud.deploy()

Deployment started in run 4 of dev-aws-us-east-1c for vespa-team.feedperformancecloud. This may take a few minutes the first time.
INFO    [07:48:08]  Deploying platform version 8.342.20 and application dev build 4 for dev-aws-us-east-1c of default ...
INFO    [07:48:08]  Using CA signed certificate version 1
INFO    [07:48:09]  Using 1 nodes in container cluster 'feedperformancecloud_container'
INFO    [07:48:14]  Session 289370 for tenant 'vespa-team' prepared and activated.
INFO    [07:48:18]  ######## Details for all nodes ########
INFO    [07:48:23]  h88962a.dev.aws-us-east-1c.vespa-external.aws.oath.cloud: expected to be UP
INFO    [07:48:23]  --- platform vespa/cloud-tenant-rhel8:8.342.20 <-- :
INFO    [07:48:23]  --- storagenode on port 19102 has not started 
INFO    [07:48:23]  --- searchnode on port 19107 has not started 
INFO    [07:48:23]  --- distributor on port 19111 has not started 
INFO    [07:48:23]  --- metricsproxy-container on port 19092 has not started 
INFO    [07

## Preparing the data

In this example we use [HF Datasets](https://huggingface.co/docs/datasets/index) library to stream the
["Cohere/wikipedia-2023-11-embed-multilingual-v3"](https://huggingface.co/datasets/Cohere/wikipedia-2023-11-embed-multilingual-v3) dataset and index in our newly deployed Vespa instance.

The dataset contains wikipedia-pages, and their corresponding embeddings.

> For this exploration we will use the `id` , `text` and `embedding`-fields

The following uses the [stream](https://huggingface.co/docs/datasets/stream) option of datasets to stream the data without
downloading all the contents locally.

The `map` functionality allows us to convert the
dataset fields into the expected feed format for `pyvespa` which expects a dict with the keys `id` and `fields`:

`{ "id": "vespa-document-id", "fields": {"vespa_field": "vespa-field-value"}}`


In [28]:
from datasets import load_dataset

dataset = load_dataset(
    "Cohere/wikipedia-2023-11-embed-multilingual-v3",
    "simple",
    split="train",
    streaming=True,
)

## Utility function to stream different number of documents


In [39]:
def get_dataset(n_docs: int = 1000):
    return (
        dataset.map(
            lambda x: {
                "id": x["_id"] + "-iter",
                "fields": {"text": x["text"], "embedding": x["emb"]},
            }
        )
        .select_columns(["id", "fields"])
        .take(n_docs)
    )

### A dataclass to store the parameters and results of the different feeding methods


In [40]:
from dataclasses import dataclass
from typing import Callable, Optional, Iterable, Dict


@dataclass
class FeedParams:
    name: str
    num_docs: int
    max_connections: int
    function_name: str
    max_workers: Optional[int] = None
    max_queue_size: Optional[int] = None
    num_concurrent_requests: Optional[int] = None


@dataclass
class FeedResult(FeedParams):
    feed_time: Optional[float] = None

### A common callback function to notify if something goes wrong


In [41]:
from vespa.io import VespaResponse


def callback(response: VespaResponse, id: str):
    if not response.is_successful():
        print(
            f"Failed to feed document {id} with status code {response.status_code}: Reason {response.get_json()}"
        )

### Defining our feeding functions


In [42]:
import time
import asyncio


def feed_sync(params: FeedParams, data: Iterable[Dict]) -> FeedResult:
    start_time = time.time()
    with app.syncio(connections=params.max_connections):
        for doc in data:
            app.feed_data_point(
                data_id=doc["id"],
                fields=doc["fields"],
                schema="doc",
                namespace="pyvespa-feed",
                callback=callback,
            )
    end_time = time.time()
    return FeedResult(
        **params.__dict__,
        feed_time=end_time - start_time,
    )


async def feed_async(params: FeedParams, data: Iterable[Dict]) -> FeedResult:
    start_time = time.time()
    tasks = []
    # We use a semaphore to limit the number of concurrent requests, this is useful to avoid
    # running into memory issues when feeding a large number of documents
    semaphore = asyncio.Semaphore(params.num_concurrent_requests)

    async with app.asyncio(
        connections=params.max_connections, total_timeout=params.num_docs // 10
    ) as async_app:
        for doc in data:
            async with semaphore:
                task = asyncio.create_task(
                    async_app.feed_data_point(
                        data_id=doc["id"],
                        fields=doc["fields"],
                        schema="doc",
                        namespace="pyvespa-feed",
                        timeout=10,
                    )
                )
                tasks.append(task)

        await asyncio.wait(tasks, return_when=asyncio.ALL_COMPLETED)

    end_time = time.time()
    return FeedResult(
        **params.__dict__,
        feed_time=end_time - start_time,
    )


def feed_iterable(params: FeedParams, data: Iterable[Dict]) -> FeedResult:
    start = time.time()
    app.feed_iterable(
        data,
        schema="doc",
        namespace="pyvespa-feed",
        operation_type="feed",
        max_queue_size=params.max_queue_size,
        max_workers=params.max_workers,
        max_connections=params.max_connections,
        callback=callback,
    )
    end = time.time()
    sync_feed_time = end - start
    return FeedResult(
        **params.__dict__,
        feed_time=sync_feed_time,
    )

## Defining our hyperparameters


In [57]:
from itertools import product

# We will only run for 1000 documents here as notebook is run as part of CI.
# But you will see some plots when run with 100k documents as well.

num_docs = [1000, 10_000]

params_by_function = {
    "feed_sync": {
        "num_docs": num_docs,
        "max_connections": [16, 64],
    },
    "feed_async": {
        "num_docs": num_docs,
        "max_connections": [16],
        "num_concurrent_requests": [1000],
    },
    "feed_iterable": {
        "num_docs": num_docs,
        "max_connections": [64],
        "max_workers": [64],
        "max_queue_size": [1000],
    },
}

feed_params = []
# Create one FeedParams instance of each permutation
for func, parameters in params_by_function.items():
    print(f"Function: {func}")
    keys, values = zip(*parameters.items())
    for combination in product(*values):
        settings = dict(zip(keys, combination))
        print(settings)
        feed_params.append(
            FeedParams(
                name=f"{settings['num_docs']}_{settings['max_connections']}_{settings.get('max_workers', 0)}_{func}",
                function_name=func,
                **settings,
            )
        )
    print("\n")  # Just to add space between different functions

Function: feed_sync
{'num_docs': 1000, 'max_connections': 16}
{'num_docs': 1000, 'max_connections': 64}
{'num_docs': 10000, 'max_connections': 16}
{'num_docs': 10000, 'max_connections': 64}


Function: feed_async
{'num_docs': 1000, 'max_connections': 16, 'num_concurrent_requests': 1000}
{'num_docs': 10000, 'max_connections': 16, 'num_concurrent_requests': 1000}


Function: feed_iterable
{'num_docs': 1000, 'max_connections': 64, 'max_workers': 64, 'max_queue_size': 1000}
{'num_docs': 10000, 'max_connections': 64, 'max_workers': 64, 'max_queue_size': 1000}




In [58]:
print(f"Total number of feed_params: {len(feed_params)}")

Total number of feed_params: 8


Now, we will need a way to retrieve the callable function from the function name.


In [59]:
# Get reference to function from string name
def get_func_from_str(func_name: str) -> Callable:
    return globals()[func_name]

### Function to clean up after each feed

For a fair comparison, we will delete the data before feeding it again.


In [60]:
from typing import Iterable, Dict


def delete_data(data: Iterable[Dict]):
    app.feed_iterable(
        iter=data,
        schema="doc",
        namespace="pyvespa-feed",
        operation_type="delete",
        callback=callback,
    )

## Main experiment loop


The line below is used to make the code run in Jupyter, as it is already running an event loop


In [61]:
import nest_asyncio

nest_asyncio.apply()

In [62]:
results = []
for params in feed_params:
    print("-" * 50)
    print("Starting feed with params:")
    print(params)
    data = get_dataset(params.num_docs)
    if "async" not in params.function_name:
        if "feed_sync" in params.function_name:
            print("Skipping feed_sync")
            continue
        feed_result = get_func_from_str(params.function_name)(params=params, data=data)
    else:
        feed_result = asyncio.run(
            get_func_from_str(params.function_name)(params=params, data=data)
        )
    print(feed_result.feed_time)
    results.append(feed_result)
    print("Deleting data")
    delete_data(data)

--------------------------------------------------
Starting feed with params:
FeedParams(name='1000_16_0_feed_sync', num_docs=1000, max_connections=16, function_name='feed_sync', max_workers=None, max_queue_size=None, num_concurrent_requests=None)
Skipping feed_sync
--------------------------------------------------
Starting feed with params:
FeedParams(name='1000_64_0_feed_sync', num_docs=1000, max_connections=64, function_name='feed_sync', max_workers=None, max_queue_size=None, num_concurrent_requests=None)
Skipping feed_sync
--------------------------------------------------
Starting feed with params:
FeedParams(name='10000_16_0_feed_sync', num_docs=10000, max_connections=16, function_name='feed_sync', max_workers=None, max_queue_size=None, num_concurrent_requests=None)
Skipping feed_sync
--------------------------------------------------
Starting feed with params:
FeedParams(name='10000_64_0_feed_sync', num_docs=10000, max_connections=64, function_name='feed_sync', max_workers=None

In [20]:
# Create a pandas DataFrame with the results
import pandas as pd

df = pd.DataFrame([result.__dict__ for result in results])
df["requests_per_second"] = df["num_docs"] / df["feed_time"]
df

,name,num_docs,max_connections,function_name,max_workers,max_queue_size,num_concurrent_requests,feed_time,requests_per_second
0,10000_16_0_feed_sync,10000,16,feed_sync,NaN,NaN,NaN,49.325479,202.734981
1,10000_64_0_feed_sync,10000,64,feed_sync,NaN,NaN,NaN,39.845676,250.968259
2,10000_16_0_feed_async,10000,16,feed_async,NaN,NaN,1000.0,32.393786,308.701182
3,10000_16_0_feed_async,10000,16,feed_async,NaN,NaN,10000.0,32.446093,308.203517
4,10000_64_0_feed_async,10000,64,feed_async,NaN,NaN,1000.0,32.581518,306.922469
5,10000_64_0_feed_async,10000,64,feed_async,NaN,NaN,10000.0,32.413599,308.512488
6,10000_64_16_feed_iterable,10000,64,feed_iterable,16.0,1000.0,NaN,23.700497,421.932082
7,10000_64_16_feed_iterable,10000,64,feed_iterable,16.0,10000.0,NaN,32.672392,306.068807
8,10000_64_64_feed_iterable,10000,64,feed_iterable,64.0,1000.0,NaN,20.216159,494.653810
9,10000_64_64_feed_iterable,10000,64,feed_iterable,64.0,10000.0,NaN,20.259803,493.588208


In [21]:
import plotly.express as px


def plot_performance(df: pd.DataFrame):
    # Create a scatter plot with logarithmic scale for both axes using Plotly Express
    fig = px.scatter(
        df,
        x="num_docs",
        y="requests_per_second",
        color="function_name",  # Defines color based on different functions
        log_x=True,  # Set x-axis to logarithmic scale
        log_y=False,  # If you also want the y-axis in logarithmic scale, set this to True
        title="Performance: Requests per Second vs. Number of Documents",
        labels={  # Customizing axis labels
            "num_docs": "Number of Documents",
            "requests_per_second": "Requests per Second",
            "max_workers": "max_workers",
            "max_queue_size": "max_queue_size",
            "max_connections": "max_connections",
            "num_concurrent_requests": "num_concurrent_requests",
        },
        template="plotly_white",  # This sets the style to a white background, adhering to Tufte's minimalist principles
        hover_data=[
            "max_workers",
            "max_queue_size",
            "max_connections",
            "num_concurrent_requests",
        ],  # Additional information to show on hover
    )

    # Update layout for better readability, similar to 'talk' context in Seaborn
    fig.update_layout(
        font=dict(
            size=16,  # Adjusting font size for better visibility, similar to 'talk' context
        ),
        legend_title_text="Function Details",  # Custom legend title
        legend=dict(
            title_font_size=16,
            x=800,  # Adjusting legend position similar to bbox_to_anchor in Matplotlib
            xanchor="auto",
            y=1,
            yanchor="auto",
        ),
        width=800,  # Adjusting width of the plot
    )
    fig.update_xaxes(
        tickvals=[1000, 10000, 100000],  # Set specific tick values
        ticktext=["1k", "10k", "100k"],  # Set corresponding tick labels
    )

    fig.update_traces(
        marker=dict(size=12, opacity=0.7)
    )  # Adjust marker size and opacity
    # Show plot
    fig.show()
    # Save plot as HTML file
    fig.write_html("performance.html")


plot_performance(df)

Here is the corresponding plot when run with 1k, 10k, and 100k documents:


![image](../../_static/feed_performance.png)


Interesting. Let's try to summarize the insights we got from this experiment:

- The `feed_sync` method is the slowest, and does not benefit much from increasing 'max_connections'.
  As there is no concurrency, and each request is blocking, this will be bound by the network latency, which in many cases will be a lot higher than when running against a local VespaDocker instance. For example, if you have 100ms latency against your Vespa instance, you can only feed 10 documents per second using the `VespaSync` method.
- The `feed_async` method is the fastest, and benefits slightly from increasing 'max_connections' regardless of the number of documents. This method is non-blocking, and will likely be even more beneficial when running against a remote Vespa instance, such as [Vespa Cloud](https://cloud.vespa.ai/), when network latency is higher.
- The `feed_iterable` performance is somewhere in between the other two methods, and benefits a lot from increasing `num_workers` when the number of documents increases.

We have not looked at multiprocessing, but there is definitively room to utilize more cores to improve further upon these results.
But there is one alternative that it is interesting to compare against, the Vespa CLI.


## Feeding with Vespa CLI

[Vespa CLI](https://docs.vespa.ai/en/vespa-cli) is a command-line interface for interacting with Vespa.

Among many useful features are a `vespa feed` command that is the recommended way of feeding large datasets into Vespa.
This is optimized for high feeding performance, and it will be interesting to get a feel for how performant feeding to a local Vespa instance is using the CLI.

Note that comparing feeding with the CLI is not entirely fair, as the CLI relies on prepared data files, while the pyvespa methods are streaming the data before feeding it.


## Prepare the data for Vespa CLI

Vespa CLI can feed data from either many .json files or a single .jsonl file with many documents.
The json format needs to be in the following format:

```json
{
  "put": "id:namespace:document-type::document-id",
  "fields": {
    "field1": "value1",
    "field2": "value2"
  }
}
```

Where, `put` is the document operation in this case. Other allowed operations are `get`, `update` and `remove`.

For reference, see https://docs.vespa.ai/en/vespa-cli#cheat-sheet

### Getting the datasets as .jsonl files

Now, let`s save the dataset to 3 different jsonl files of 1k, 10k, and 100k documents.


In [22]:
for n in num_docs:
    print(f"Getting dataset with {n} docs...")
    # First, let's load the dataset in non-streaming mode this time, as we want to save it to a jsonl file
    dataset_cli = load_dataset(
        "Cohere/wikipedia-2023-11-embed-multilingual-v3",
        "simple",
        split=f"train[:{n}]",  # Notice the slicing here, see https://huggingface.co/docs/datasets/loading#slice-splits
        streaming=False,
    )
    # Map to the format expected by the CLI.
    # Note that this differs a little bit from the format expected by the Python API.
    dataset_cli = dataset_cli.map(
        lambda x: {
            "put": f"id:pyvespa-feed:doc::{x['_id']}-json",
            "fields": {"text": x["text"]},
        }
    ).select_columns(["put", "fields"])
    # Save to a jsonl file
    assert len(dataset_cli) == n
    dataset_cli.to_json(f"vespa_feed-{n}.json", orient="records", lines=True)

Getting dataset with 10000 docs...


Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Let's look at the first line of one of the saved files to verify the format.


In [23]:
from pprint import pprint
import json

with open("vespa_feed-1000.json", "r") as f:
    sample = f.readline()
    pprint(json.loads(sample))

{'fields': {'text': 'April (Apr.) is the fourth month of the year in the '
                    'Julian and Gregorian calendars, and comes between March '
                    'and May. It is one of the four months to have 30 days.'},
 'put': 'id:pyvespa-feed:doc::20231101.simple_1_0-json'}


Ok, now we are ready to feed the data using Vespa CLI.
We also want to capture the output of feed statistics for each file.


In [24]:
cli_results = {}
for n in num_docs:
    print(f"Feeding {n} docs...")
    output_list = !vespa feed vespa_feed-{n}.json
    results = json.loads("".join(output_list))
    pprint(results)
    cli_results[n] = results

Feeding 10000 docs...
{'feeder.error.count': 0,
 'feeder.inflight.count': 0,
 'feeder.ok.count': 10000,
 'feeder.ok.rate': 10000.0,
 'feeder.operation.count': 10000,
 'feeder.seconds': 0.632,
 'http.exception.count': 0,
 'http.request.MBps': 2.906,
 'http.request.bytes': 2905878,
 'http.request.count': 10000,
 'http.response.MBps': 1.317,
 'http.response.bytes': 1317226,
 'http.response.code.counts': {'200': 10000},
 'http.response.count': 10000,
 'http.response.error.count': 0,
 'http.response.latency.millis.avg': 76,
 'http.response.latency.millis.max': 123,
 'http.response.latency.millis.min': 28}


In [25]:
cli_results

{10000: {'feeder.operation.count': 10000,
  'feeder.seconds': 0.632,
  'feeder.ok.count': 10000,
  'feeder.ok.rate': 10000.0,
  'feeder.error.count': 0,
  'feeder.inflight.count': 0,
  'http.request.count': 10000,
  'http.request.bytes': 2905878,
  'http.request.MBps': 2.906,
  'http.exception.count': 0,
  'http.response.count': 10000,
  'http.response.bytes': 1317226,
  'http.response.MBps': 1.317,
  'http.response.error.count': 0,
  'http.response.latency.millis.min': 28,
  'http.response.latency.millis.avg': 76,
  'http.response.latency.millis.max': 123,
  'http.response.code.counts': {'200': 10000}}}

In [26]:
# Let's add the CLI results to the DataFrame
df_cli = pd.DataFrame(
    [
        {
            "name": f"{n}_cli",
            "num_docs": n,
            "max_connections": "unknown",
            "function_name": "cli",
            "max_workers": "unknown",
            "max_queue_size": "n/a",
            "feed_time": result["feeder.seconds"],
        }
        for n, result in cli_results.items()
    ]
)
df_cli["requests_per_second"] = df_cli["num_docs"] / df_cli["feed_time"]
df_cli

,name,num_docs,max_connections,function_name,max_workers,max_queue_size,feed_time,requests_per_second
0,10000_cli,10000,unknown,cli,unknown,n/a,0.632,15822.78481


In [27]:
df_total = pd.concat([df, df_cli])

plot_performance(df_total)

And here is the corresponding plot when run with 1k, 10k, and 100k documents:


![image](../../_static/feed_performance-cli.png)


As you can tell, the CLI is orders of magnitude faster.


## Conclusion


- Prefer to use the CLI if you care about performance. 🚀
- If you are using Python, prefer the async method, as it is the fastest way to feed data using `pyvespa`.


## Cleanup


In [28]:
app.delete()

## Next steps

Check out some of the other
[examples](https://pyvespa.readthedocs.io/en/latest/examples.html) in the documentation.
